# Benchmarking Setup

To prove the robustness of TD2C, we compare it against state-of-the-art methods from different families:

| Family | Method | Library |
| :--- | :--- | :--- |
| **Time Series** | Multivariate Granger | `statsmodels` |
| **Constraint-based** | PCMCI | `tigramite` |
| **Noise-based** | VarLiNGAM | `lingam` |
| **Score-based** | DYNOTEARS | `causalnex` |

### Running a Benchmark
The `td2c` package provides a unified interface to run these diverse methods.

In [ ]:
from td2c.benchmark import VAR, Granger, PCMCI, DYNOTEARS, D2CWrapper
from td2c.data_generation.builder import TSBuilder
from td2c.descriptors import DataLoader

# 1. Generate a Test Time Series
builder = TSBuilder(n_variables=4, maxlags=2, observations_per_time_series=200, time_series_per_process=1, verbose=False)
builder.build()
loader = DataLoader(n_variables=4, maxlags=2)
loader.from_tsbuilder(builder)
observations = loader.get_original_observations() # List of arrays

# 2. Run Competitors (if installed)
results = {}

try:
    print("Running Granger...")
    granger = Granger(ts_list=observations, maxlags=2, n_jobs=1)
    granger.run()
    results['Granger'] = granger.get_causal_dfs()[0]
except ImportError:
    print("Statsmodels not installed.")

# (Assuming model `clf` is available from previous steps or loaded)
print("Running TD2C...")
# We use the D2CWrapper to handle the pipeline automatically
td2c = D2CWrapper(
    ts_list=loader.get_observations(), # Note: TD2C takes lagged obs
    model=clf, # The trained BalancedRandomForest
    n_variables=4,
    maxlags=2,
    mb_estimator="ts",
    n_jobs=1
)
td2c.run()
results['TD2C'] = td2c.get_causal_dfs()[0]

print("Benchmarking Complete.")